# Multi step model (vector output approach)

Download zipfile from https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip?dl=0 and store in the data folder.

In this notebook, we demonstrate how to:
- prepare time series data for training a RNN forecasting model
- get data in the required shape for the keras API
- implement a RNN model in keras to predict the next 24 steps ahead (time *t+1* to *t+24*) in the time series. This model uses recent values of temperature and load as the model input. The model will be trained to output a vector, the elements of which are ordered predictions for future time steps.
- enable early stopping to reduce the likelihood of model overfitting
- evaluate the model on a test dataset

The data in this example is taken from the GEFCom2014 forecasting competition<sup>1</sup>. It consists of 3 years of hourly electricity load and temperature values between 2012 and 2014. The task is to forecast future values of electricity load.

<sup>1</sup>Tao Hong, Pierre Pinson, Shu Fan, Hamidreza Zareipour, Alberto Troccoli and Rob J. Hyndman, "Probabilistic energy forecasting: Global Energy Forecasting Competition 2014 and beyond", International Journal of Forecasting, vol.32, no.3, pp 896-913, July-September, 2016.

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from collections import UserDict
from glob import glob
%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)

In [2]:
%run -i common/load_data.py
%run -i common/mape.py
%run -i common/TimeSeriesTensor.py
%run -i common/create_evaluation_df.py

Load data into Pandas dataframe

In [3]:
if not os.path.exists(os.path.join('data', 'energy.csv')):
    %run common/extract_data.py
energy = load_data()
energy.head()

,load,temp
2012-01-01 00:00:00,"2,698.00",32.00
2012-01-01 01:00:00,"2,558.00",32.67
2012-01-01 02:00:00,"2,444.00",30.00
2012-01-01 03:00:00,"2,402.00",31.00
2012-01-01 04:00:00,"2,403.00",32.00


In [4]:
valid_start_dt = '2014-09-01 00:00:00'
test_start_dt = '2014-11-01 00:00:00'

T = 6
HORIZON = 24
N_EXPERIMENTS = 10

In [5]:
train = energy.copy()[energy.index < valid_start_dt][['load', 'temp']]

In [6]:
from sklearn.preprocessing import MinMaxScaler

y_scaler = MinMaxScaler()
y_scaler.fit(train[['load']])

X_scaler = MinMaxScaler()
train[['load', 'temp']] = X_scaler.fit_transform(train)

Use the TimeSeriesTensor convenience class to:
1. Shift the values of the time series to create a Pandas dataframe containing all the data for a single training example
2. Discard any samples with missing values
3. Transform this Pandas dataframe into a numpy array of shape (samples, time steps, features) for input into Keras

The class takes the following parameters:

- **dataset**: original time series
- **H**: the forecast horizon
- **tensor_structure**: a dictionary discribing the tensor structure in the form { 'tensor_name' : (range(max_backward_shift, max_forward_shift), [feature, feature, ...] ) }
- **freq**: time series frequency
- **drop_incomplete**: (Boolean) whether to drop incomplete samples

In [7]:
tensor_structure = {'X':(range(-T+1, 1), ['load', 'temp'])}
train_inputs = TimeSeriesTensor(train, 'load', HORIZON, tensor_structure)

In [8]:
train_inputs.dataframe.head(3)

tensor              target                                              ...   \
feature                  y                                              ...    
time step              t+1  t+2  t+3  t+4  t+5  t+6  t+7  t+8  t+9 t+10 ...    
2012-01-01 05:00:00   0.18 0.23 0.29 0.35 0.37 0.37 0.37 0.36 0.35 0.36 ...    
2012-01-01 06:00:00   0.23 0.29 0.35 0.37 0.37 0.37 0.36 0.35 0.36 0.46 ...    
2012-01-01 07:00:00   0.29 0.35 0.37 0.37 0.37 0.36 0.35 0.36 0.46 0.54 ...    

tensor                 X                                               
feature             load                temp                           
time step            t-3  t-2  t-1    t  t-5  t-4  t-3  t-2  t-1    t  
2012-01-01 05:00:00 0.14 0.13 0.13 0.15 0.42 0.43 0.40 0.41 0.42 0.41  
2012-01-01 06:00:00 0.13 0.13 0.15 0.18 0.43 0.40 0.41 0.42 0.41 0.40  
2012-01-01 07:00:00 0.13 0.15 0.18 0.23 0.40 0.41 0.42 0.41 0.40 0.39  

[3 rows x 36 columns]

In [9]:
X_train = train_inputs.dataframe.as_matrix()[:,HORIZON:]
X_train.shape

(23347, 12)

In [10]:
train_inputs['target']

array([[0.18, 0.23, 0.29, ..., 0.1 , 0.12, 0.16],
       [0.23, 0.29, 0.35, ..., 0.12, 0.16, 0.23],
       [0.29, 0.35, 0.37, ..., 0.16, 0.23, 0.3 ],
       ...,
       [0.33, 0.25, 0.19, ..., 0.61, 0.58, 0.51],
       [0.25, 0.19, 0.16, ..., 0.58, 0.51, 0.43],
       [0.19, 0.16, 0.14, ..., 0.51, 0.43, 0.34]])

Construct validation set (keeping T hours from the training set in order to construct initial features)

In [11]:
look_back_dt = dt.datetime.strptime(valid_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
valid = energy.copy()[(energy.index >=look_back_dt) & (energy.index < test_start_dt)][['load', 'temp']]
valid[['load', 'temp']] = X_scaler.transform(valid)
valid_inputs = TimeSeriesTensor(valid, 'load', HORIZON, tensor_structure)
X_valid = valid_inputs.dataframe.as_matrix()[:,HORIZON:]
X_valid.shape

(1440, 12)

Construct test set (keeping T hours from the validation set in order to construct initial features)

In [12]:
look_back_dt = dt.datetime.strptime(test_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
test = energy.copy()[test_start_dt:][['load', 'temp']]
test[['load', 'temp']] = X_scaler.transform(test)
test_inputs = TimeSeriesTensor(test, 'load', HORIZON, tensor_structure)
X_test = test_inputs.dataframe.as_matrix()[:,HORIZON:]
X_test.shape

(1435, 12)

## Implement the RNN

In [13]:
from keras.models import Model, Sequential
from keras.layers import GRU, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
LATENT_DIM = 5
BATCH_SIZE = 32
EPOCHS = 50

In [15]:
def get_model():
    model = Sequential()
    model.add(Dense(LATENT_DIM, activation="relu", input_shape=(2*T,)))
    model.add(Dense(HORIZON))
    model.compile(optimizer='RMSprop', loss='mse')
    
    return model

In [18]:
mapes = np.empty(N_EXPERIMENTS)
for i in range(N_EXPERIMENTS):
    
    # Initialize the model
    model = get_model()
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
    best_val = ModelCheckpoint('model_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)
    
    # Train the model
    history = model.fit(X_train,
                        train_inputs['target'],
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(X_valid, valid_inputs['target']),
                        callbacks=[earlystop, best_val],
                        verbose=0)
    
    # load the model with the smallest MAPE
    best_epoch = np.argmin(np.array(history.history['val_loss']))+1
    model.load_weights("model_{:02d}.h5".format(best_epoch))
    
    predictions = model.predict(X_test)
    
    # Compute MAPE for each forecast horizon
    eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
    eval_df['APE'] = (eval_df['prediction'] - eval_df['actual']).abs() / eval_df['actual']
    
    # Compute MAPE across all predictions
    mapes[i] = mape(eval_df['prediction'], eval_df['actual'])
    print('{0:.4f}'.format(mapes[i]))
    
    for f in glob('model_*.h5'):
        os.remove(f)

0.0764
0.1158
0.0701
0.0914
0.0776
0.0778
0.0915
0.0728
0.0921
0.0754


In [19]:
result = 'Mean MAPE = {0:.4f} +/- {1:.4f}'.format(np.mean(mapes), np.std(mapes)/np.sqrt(N_EXPERIMENTS))
print(result)

Mean MAPE = 0.0841 +/- 0.0041
